# Store Sales Average Model
---
Notebook ini merupakan studi mengenai hubungan antara Jenis Bangunan, Kelas Bangunan, Lokasi Kabupaten/Kota dan Luas Area Toko dalam men-generate penjualan per bulan di PT Prestasi Retail Innovation.
- Jenis Bangunan: Merupakan satu dari tiga tipe bangunan diantaranya Mall (M), Ruko (R) dan Bangunan Sendiri (S)
- Kelas Bangunan: Merupakan kelas dari tipe bangunan tersebut dalam men-generate sales. Misalkan Mall Grand Indonesia adalah Mall Kelas 1 (M1) sedangkan Gandaria City adalah Mall Kelas 4 (M4). Berikut adalah Kelas Bangunan yang ada.
  - Mall Kelas 1 (M1)
  - Mall Kelas 2 (M2)
  - Mall Kelas 3 (M3)
  - Mall Kelas 4 (M4)
  - Mall Kelas 5 (M5)
  - Ruko Kelas 1 (R1)
  - Ruko Kelas 2 (R2)
  - Ruko Kelas 3 (R3)
  - Ruko Kelas 4 (R4)
  - Ruko Kelas 5 (R5)
  - Bangunan Sendiri Kelas 1 (S1)
  - Bangunan Sendiri Kelas 2 (S2)
  - Bangunan Sendiri Kelas 3 (S3)
  - Bangunan Sendiri Kelas 4 (S4)
  - Bangunan Sendiri Kelas 5 (S5)
- Lokasi Kabupaten/Kota adalah lokasi geografis dari toko
- Luas Area Toko adalah luas meter persegi dari toko
---
Studi ini akan menggunakan Regresi Linear dalam merumuskan nilai Average Sales ($y$) yang dipengaruhi oleh variabel - variabel independen lainnya seperti Jenis Bangunan $x{_1}$, Kelas Bangunan $x{_2}$, Lokasi Kabupaten/Kota ($x{_3}$) dan Luas Area Toko ($x{_4}$). Berikut adalah formulasi Regresi Linear untuk permasalahan tersebut:  
  
$
y = ax{_1} + bx{_2} + cx{_3} + dx{_4} + e
$  
  
Dimana:  
$y$ = Prediksi Average Sales  
$x{_1}$ = Variabel independen mewakili Jenis Bangunan  
$x{_2}$ = Variabel independen mewakili Kelas Bangunan  
$x{_3}$ = Variabel independen mewakili Lokasi Kabupaten/Kota  
$x{_4}$ = Variabel independen mewakili Luas Area Toko  
$a$ = Koefisien variabel independen $x{_1}$  
$b$ = Koefisien variabel independen $x{_2}$  
$c$ = Koefisien variabel independen $x{_3}$  
$d$ = Koefisien variabel independen $x{_4}$  
$e$ = Bias dari Regresi Linear  

### Import Modul

In [ ]:
from typing import Literal
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import lightgbm
import xgboost
import tensorflow as tf
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn.objects as so
sns.set()

### Versi Tensorflow

In [ ]:
print(tf.__version__)
print(tf.config.list_physical_devices())

### Format Tampilan DataFrame

In [ ]:
pd.options.display.float_format = '{:,}'.format

## Eksplorasi Data
---

### Atribut Dasar Toko
Berikut adalah beberapa data atribut dasar toko yang saat ini dimiliki.

In [ ]:
data_toko = pd.read_excel("PRI - Store Renov Rent.xlsx", sheet_name=0, header=0)
data_toko

Karena kita tidak akan menggunakan semua kolom dalam data ini untuk kepentingan studi Store Sales Average, maka `data_toko` akan diringkas dan disusun ulang menjadi:

In [ ]:
data_toko = data_toko[["STORE CODE", "STORE NAME", "Tipe Bangunan", "Kelas Bangunan", "Kota Kabupaten 2", "Estimasi Populasi", "sqm"]]
data_toko

#### Luas Area Toko (sqm)

In [ ]:
data_luas = data_toko["sqm"]
hitung, bin = np.histogram(data_luas)
print(hitung, bin)

In [ ]:
plot = so.Plot(data_toko, "sqm")
plot.add(so.Bars(), so.Hist(), legend=True).label(title="Persebaran Toko berdasar SQM", x="Square Meters (sqm)", y="Jumlah Toko")

Berdasarkan luasnya (sqm), kita dapat melihat pada fungsi `histogram` di atas bahwa distribusi persebaran luas toko cukup normal dengan 29 Toko jatuh ke dalam kategori `sqm` di antara $75m{^2}$ sampai dengan $150m{^2}$, dengan 1 toko yang menjadi outlier dari distribusi dimana luas toko > $400m{^2}$.

#### Penjualan
---
Mengingat bahwa data *historical* yang dimiliki terbatas dari tahun 2018 sampai dengan November 2022, serta mengingat bahwa kita mengalami periode pandemi CoV-19 selama lebih dari 1 tahun, maka penulis merasa perlu untuk melakukan separasi data penjualan per bulan menggunakan flag `Pandemic`.
  
Berikut adalah sepenggal data penjualan *historical* per toko dari tahun 2018 sampai dengan November 2022 (40 baris data awal).

In [ ]:
data_penjualan = pd.read_excel("PRI - Store Renov Rent.xlsx", sheet_name="Sales", header=0)
data_penjualan

Berikut adalah grouping data penjualan *historical* yang disimpan dalam variabel `data_penjualan_by_month`.

In [ ]:
data_penjualan_by_month = data_penjualan.groupby(["EOM"]).sum(numeric_only=True)
data_penjualan_by_month

Untuk melihat trend pergerakan penjualan dalam kurun waktu ini, kita akan menggunakan `Simple Moving Average` yang akan menghitung rerata secara bergulung untuk interval waktu ke belakang (contoh: Moving Average 3 Bulan untuk Mei 2021 adalah rata - rata penjualan yang merupakan rata - rata dari penjualan di bulan Maret 2021, April 2021 dan Mei 2021).  
Namun satu `Moving Average` saja tidak dapat menggambarkan sebuah trend, karena itu kita juga akan menggunakan tambahan 2 interval `Moving Average` lainnya untuk menggambarkan trend pada jangka pendek, jangka menengah dan jangka panjang.  
Dalam studi ini kita akan menggunakan 3 `Simple Moving Average` yaitu:
* `MA3`: `Moving Average` dengan jendela periode 3 bulan ke belakang (Jangka Pendek)
* `MA6`: `Moving Average` dengan jendela periode 6 bulan ke belakang (Jangka Menengah)
* `MA12`: `Moving Average` dengan jendela periode 12 bulan atau 1 tahun ke belakang (Jangka Panjang)  
Berikut adalah `data_penjualan_by_month` dengan penambahan kolom `MA3`, `MA6` dan `MA12` yang didapat dengan menggunakan fungsi `rolling()` dari `pd.DataFrame` yang dirata-ratakan dengan fungsi `mean()`.

In [ ]:
data_penjualan_by_month["MA3"] = data_penjualan_by_month["Sales"].rolling(3).mean()
data_penjualan_by_month["MA6"] = data_penjualan_by_month["Sales"].rolling(6).mean()
data_penjualan_by_month["MA12"] = data_penjualan_by_month["Sales"].rolling(12).mean()
data_penjualan_by_month

Berikut adalah grafik penjualan *historical* dari tahun 2018 sampai dengan November 2022.

In [ ]:
garis_plot = so.Plot(data_penjualan_by_month, "EOM", "Sales")
garis_plot.add(so.Line()).label(title="Total Penjualan per Bulan (2018 - Nov 2022)", x="Tahun")

Berikut adalah grafik `Moving Average` untuk data penjualan *historical* dari tahun 2018 sampai dengan November 2022.

In [ ]:
print(data_penjualan_by_month.reset_index())

In [ ]:
data_kolom_melt = pd.melt(data_penjualan_by_month.reset_index().drop(columns=["Sales"]), id_vars='EOM', var_name="Tipe MA", value_name="Nilai")
data_kolom_melt

In [ ]:
# plt.plot(data_penjualan_by_month["MA3"], color="green", label="MA3")
# plt.plot(data_penjualan_by_month["MA6"], color="orange", label="MA6")
# plt.plot(data_penjualan_by_month["MA12"], color="red", label="MA12")
# plt.show()
fig, ax = plt.subplots()
ax.set_title("Moving Average Total Penjualan per Bulan")
ax.set_ylabel("Rata-Rata Total Penjualan per Bulan")
ax.set_xlabel("Tahun")
# Plot Moving Average
sns.lineplot(data_kolom_melt, x="EOM", y="Nilai", hue="Tipe MA")
# Region Section Pandemic
ax.fill_between(data_penjualan_by_month.index.values, 0, 28000000000, where=((data_penjualan_by_month.index.values > np.datetime64('2020-02-29')) & (data_penjualan_by_month.index.values <= np.datetime64('2021-10-31'))), color="red", alpha=0.2)
# Region Section Recovery
ax.fill_between(data_penjualan_by_month.index.values, 0, 28000000000, where=((data_penjualan_by_month.index.values >= np.datetime64('2021-11-01')) & (data_penjualan_by_month.index.values <= np.datetime64('2022-12-31'))), color="green", alpha=0.2)
plt.show()

Pada grafik di atas kita dapat melihat bahwa terdapat tren penurunan penjualan (terkonfirmasi dengan `MA3` yang turun ke bawah `MA6` dan `MA12`) pada periode Maret 2020 (`Pandemic`) dan nilai rata - rata penjualan ini bertahan cukup rendah hingga setidaknya sampai dengan bulan Oktober 2021 dan di bulan November 2021 hingga seterusnya kita dapat melihat nilai rata-rata penjualan per bulan yang meningkat (`Recovery`, terkonfirmasi dengan `MA3` yang melewati dan bertahan di atas `MA6` dan `MA12`).  
  
Oleh karena itu kita akan mengkategorikan penjualan yang terjadi diantara bulan Maret 2020 - Oktober 2021 sebagai penjualan dalam masa `Pandemic` dan lainnya sebagai penjualan `Normal`.

In [ ]:
pandemic_period = [
  np.datetime64('2020-03-31'),
  np.datetime64('2020-04-30'),
  np.datetime64('2020-05-31'),
  np.datetime64('2020-06-30'),
  np.datetime64('2020-07-31'),
  np.datetime64('2020-08-31'),
  np.datetime64('2020-09-30'),
  np.datetime64('2020-10-31'),
  np.datetime64('2020-11-30'),
  np.datetime64('2020-12-31'),
  np.datetime64('2021-01-31'),
  np.datetime64('2021-02-28'),
  np.datetime64('2021-03-31'),
  np.datetime64('2021-04-30'),
  np.datetime64('2021-05-31'),
  np.datetime64('2021-06-30'),
  np.datetime64('2021-07-31'),
  np.datetime64('2021-08-31'),
  np.datetime64('2021-09-30'),
  np.datetime64('2021-10-31'),
  ]
print(pandemic_period)

Berikut adalah pengkategorian bulan penjualan berdasarkan periode `Pandemic` dan `Normal`

In [ ]:
def status_pandemi(x):
  return "Pandemic" if x["EOM"] in pandemic_period else "Normal"

data_penjualan["Status Pandemi"] = data_penjualan.apply(lambda x: status_pandemi(x), axis=1)
data_penjualan

Pembentukan dataframe `data_penjualan_rerata` untuk lookup nilai penjualan rata-rata pada masa pandemi dan normal di dataframe `data_toko`

In [ ]:
data_penjualan_rerata = data_penjualan.groupby(['Status Pandemi', 'LocationCode']).mean(numeric_only=True)
data_penjualan_rerata

Implementasi lookup rata-rata penjualan per bulan untuk setiap toko baik pada masa pandemi maupun pada masa normal di dataframe `data_toko`

In [ ]:
def lookup_rerata(x, pandemi_status, lookup_df):
  try: 
    return sum(lookup_df.loc[pandemi_status, x['STORE CODE']].values)
  except:
    return np.NaN

data_toko["Rerata Penjualan Normal"] = data_toko.apply(lambda x: lookup_rerata(x, 'Normal', data_penjualan_rerata), axis=1) # type: ignore
data_toko["Rerata Penjualan Pandemi"] = data_toko.apply(lambda x: lookup_rerata(x, 'Pandemic', data_penjualan_rerata), axis=1) # type: ignore

data_toko

Pada dataframe `data_toko` dengan penambahan kolom `Rerata Penjualan Normal` dan `Rerata Penjualan Pandemi` kita dapat melihat bahwa nilai `Rerata Penjualan Normal` lebih besar daripada nilai `Rerata Penjualan Pandemi` untuk kesemua toko, hal ini menunjukkan bahwa kita berhasil menangkap nilai rata-rata penjualan per bulan di masa normal yang kita ekspektasikan menjadi acuan ke depannya.

Nilai pada kolom `Rerata Penjualan Normal` ini adalah nilai $y$ yang sebenarnya. Nilai $y$ yang sebenarnya ini akan menjadi acuan dalam proses pelatihan jaringan saraf tiruan untuk melihat seberapa akurat model dalam memprediksi nilai $y$ atau yang kita sebut $\hat{y}$ (*y-hat* atau prediksi y).

#### STORE CODE & KOTA KABUPATEN 2
---
Dalam membangun model prediksi, selain mempertimbangkan input dalam proses pelatihan model, kita juga harus mempertimbangkan interaksi pengguna dengan model nantinya dalam menghasilkan prediksi rata-rata penjualan per bulan.
Jika kita membayangkan pengguna melakukan input pada serangkaian form untuk mendapatkan nilai output prediksi rata-rata penjualan per bulan untuk input yang diberikan, nampaknya akan sulit jika pengguna menginput semisalkan `STORE CODE` 'FS040' atau `KOTA KABUPATEN 2` 'PALU'. Hal ini dikarenakan model akan dilatih menggunakan data pada `data_toko` yang jumlah sampelnya sangat terbatas dan tidak pernah mengenal 'FS040' atau 'PALU' sebagai salah satu input dalam proses pelatihan model.  
Oleh karena itu, kita akan melakukan modifikasi pada kedua variabel ini untuk memastikan proses pelatihan berjalan lebih umum (*general*) dan untuk memungkinkan input oleh pengguna pada model nantinya lebih umum.

##### STORE CODE
Untuk `STORE CODE`, supaya baik proses pelatihan maupun input pada model nantinya bisa berlaku secara lebih umum, kita akan menggunakan `SBU` yang diekstrak dari dua karakter pertama dalam `STORE CODE` dan untuk FO akan masuk ke dalam `SBU` 'Fisik Sport'

In [ ]:
def konversi_sbu(x):
  try:
    match x['STORE CODE'][:2]:
      case "FS" | "FO":
        return "Fisik Sport"
      case "FF":
        return "Fisik Football"
      case "OD":
        return "Our Daily Dose"
      case _:
        return np.NaN
  except:
    return np.NaN

data_toko["SBU"] = data_toko.apply(lambda x: konversi_sbu(x), axis=1) # type: ignore

# Reorder kolom
kolom = ["STORE CODE", "STORE NAME", "SBU", "Tipe Bangunan", "Kelas Bangunan", "Kota Kabupaten 2", "Estimasi Populasi", "sqm", "Rerata Penjualan Normal", "Rerata Penjualan Pandemi"]
data_toko = data_toko[kolom]
  
data_toko

##### KABUPATEN KOTA 2
Untuk `KABUPATEN KOTA 2`, kita akan melakukan grouping rentang populasi, misalkan populasi `0 - 500,000`, `500,001 - 1,000,000` dstnya. Hal ini dipandang lebih baik untuk proses pelatihan jaringan saraf tiruan model dan juga untuk implementasi prediksi model pada aplikasi ke depannya, mengingat jumlah sampel pelatihan yang sangat terbatas.  
Sebelumnya, dipandang perlu untuk melihat kardinalitas anggota dalam rentang yang terbentuk untuk memastikan distribusi yang mendekati normal.

In [ ]:
jumlah_anggota, bin = np.histogram(data_toko["Estimasi Populasi"], bins=6, range=(0, 3000000))
print(f"Kardinalitas anggota: \t{jumlah_anggota}")
print(f"Range Bin: \t\t{bin}")

Pada fungsi `histogram()` di atas kita mengelompokkan data `Estimasi Populasi` ke dalam 6 rentang dengan nilai rentang minimal dimulai dari 0 dan nilai rentang maksimal sebesar 3,000,000.

In [ ]:
fig, ax = plt.subplots()

ax.set_title("Persebaran Rentang Populasi Toko")
ax.hist(bin[:-1], weights=jumlah_anggota, range=(0, 3000000))
ax.set_ylabel("Jumlah Toko di Kota dengan Rentang Populasi")
ax.set_xlabel("Rentang Populasi")

fig.show()

Pada grafik histogram di atas kita dapat melihat bahwa persebaran data cukup normal dimana sebagian besar toko dibuka di kota dengan populasi `1,000,000 - 1,500,000` (10 Toko) dan `1,500,001 - 2,000,000` (12 Toko) penduduk.

In [ ]:
def konversi_rentang_populasi(x):
  try:
    match x['Estimasi Populasi']:
      case x if x <= 500000:
        return '0 - 500000'
      case x if x <= 1000000:
        return '500001 - 1000000'
      case x if x <= 1500000:
        return '1000001 - 1500000'
      case x if x <= 2000000:
        return '1500001 - 2000000'
      case x if x <= 2500000:
        return '2000001 - 2500000'
      case _:
        return '> 2500000'
  except:
    return

data_toko['Rentang Populasi'] = data_toko.apply(lambda x: konversi_rentang_populasi(x), axis=1) # type: ignore

# Reorder kolom
kolom = ["STORE CODE", "STORE NAME", "SBU", "Tipe Bangunan", "Kelas Bangunan", "Kota Kabupaten 2", "Estimasi Populasi", "Rentang Populasi", "sqm", "Rerata Penjualan Normal", "Rerata Penjualan Pandemi"]
data_toko = data_toko[kolom]

data_toko

## Konversi Data Categorical ke dalam Label Encoder
---
Dengan dua perubahan pada sub-bagian sebelumnya maka fungsi regresi linear dapat digambarkan ulang sebagai berikut:  
  
$
{Rerata Penjualan Normal} = a \cdot {SBU} + b \cdot {Kelas Bangunan} + c \cdot {Luas Area} + d \cdot {Rentang Populasi} + e
$ 
   
Dikarenakan `SBU`, `Kelas Bangunan` dan `Rentang Populasi` merupakan tipe data *categorical*, sedangkan pelatihan jaringan saraf tiruan untuk sebuah model memerlukan semua input dalam bentuk numerik, maka kita akan melakukan konversi pada ketiga data tersebut menjadi numerik.  
Dilihat dari jenis datanya, `SBU` merupakan data *categorical nominal*, `Rentang Populasi` merupakan *categorical ordinal*, sedangkan `Kelas Bangunan` meski sekilas nampak seperti *categorical ordinal* dengan susunan hierarki dan memiliki bobot, namun hierarki dan bobot ini menjadi ambigu ketika kita beralih dari M5 ke R1 atau hierarki dan bobot yang tidak jelas antara R1 dan S1 mengenai mana yang lebih memiliki bobot, oleh karena itu untuk menjaga *prudentiality* dari model maka kita akan mengkategorikan `Kelas Bangunan` sebagai *categorical nominal* yang hanya merupakan label tanpa bobot antar kategori di dalamnya.  
Untuk data *categorical nominal* kita akan menerapkan proses *One Hot Encoding* untuk menerapakan pelabelan numerik tanpa susunan maupun bobot dan untuk data *categorical ordinal* kita akan menggunakan *Ordinal Encoding*.

In [ ]:
# One Hot Encoding SBU dan Kelas Bangunan
ohe = OneHotEncoder(sparse_output=False)
sbu_encoded = ohe.fit_transform(data_toko["SBU"].values.reshape(-1, 1)) # type: ignore
data_sbu_encoded = pd.DataFrame(sbu_encoded, columns=ohe.get_feature_names_out(["SBU"]))
kelas_bangunan_encoded = ohe.fit_transform(data_toko["Kelas Bangunan"].values.reshape(-1, 1)) # type: ignore
data_kelas_bangunan_encoded = pd.DataFrame(kelas_bangunan_encoded, columns=ohe.get_feature_names_out(["Kelas Bangunan"]))
print(f"Data SBU setelah proses One Hot Encoding: \n{data_sbu_encoded}")
print(f"Data Kelas Bangunan setelah proses One Hot Encoding: \n{data_kelas_bangunan_encoded.to_string()}")

# Ordinal Encoding Rentang Populasi
oe = OrdinalEncoder()
rentang_populasi_encoded = oe.fit_transform(data_toko["Rentang Populasi"].values.reshape(-1, 1)) # type: ignore
data_rentang_populasi_encoded = pd.DataFrame(rentang_populasi_encoded, columns=["Rentang Populasi Encoded"])
print(f"Data Rentang Populasi setelah proses Ordinal Encoding: \n{data_rentang_populasi_encoded.to_string()}")

In [ ]:
# Copy data_toko
data_model = data_toko.copy()

# Penambahan data yang sudah melalui proses encoding
data_model = pd.concat([data_model, data_sbu_encoded, data_kelas_bangunan_encoded, data_rentang_populasi_encoded], axis=1)

# data_model sebelum drop data yang tidak digunakan dalam pelatihan
data_model

In [ ]:
# Drop data yang tidak dipergunakan dalam pelatihan
data_model = data_model.drop(["STORE CODE", "STORE NAME", "SBU", "Tipe Bangunan", "Kelas Bangunan", "Kota Kabupaten 2", "Estimasi Populasi", "Rentang Populasi", "Rerata Penjualan Pandemi"], axis=1)
# Drop data dengan Rerata Penjualan Normal yang NaN
data_model.dropna(subset="Rerata Penjualan Normal", axis=0, inplace=True)

# Data untuk proses pelatihan model
data_model

Dan berikut adalah distribusi variabel independen terkait dengan variable independen lainnya.

In [ ]:
sns.pairplot(data=data_model, diag_kind='kde')

## Scaling dan Pembentukan Data Train Test
---

### Scaling
Merujuk kepada nilai dalam data yang dimiliki, kita sebenarnya hanya memiliki 1 fitur (kolom) dengan nilai numerik, yaitu kolom `sqm` sedangkan sisanya merupakan kategori yang di-encode baik secara One Hot Encoding maupun Label Encoding. Meski *scale* antara `sqm` dan `Rerata Penjualan Normal` cukup jauh, namun mengingat `sqm` adalah bagian dari fitur ($x$) sedangkan `Rerata Penjualan Normal` adalah target ($y$), maka kita tidak perlu melakukan *scaling* pada fitur ($x$).

### Pembentukan Data Train Test

#### Data X y
Data X yang akan dipergunakan sebagai fitur adalah semua kolom pada `data_model` terkecuali kolom `Rerata Penjualan Normal` yang akan menjadi Data y.

In [ ]:
X = data_model.drop("Rerata Penjualan Normal", axis=1)
y = data_model["Rerata Penjualan Normal"]
print("Data X:")
print(X.to_string())
print("\nData y:")
print(y.to_string())

#### Data Train Test
Pembagian data train dan test adalah dengan rasio data test sebesar 0.2 dari total data, menggunakan random_state yang di-set pada 11

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12, shuffle=True)
print(f"X_train:\n{X_train.to_string()}\n") # type: ignore
print(f"y_train:\n{y_train.to_string()}\n") # type: ignore
print(f"X_test:\n{X_test.to_string()}\n") # type: ignore
print(f"y_test:\n{y_test.to_string()}") # type: ignore

## Model
---

### Pembentukan Model
Pada bagian ini kita akan coba untuk membuat beberapa model yang akan dipergunakan dalam pelatihan nantinya. Pelatihan model akan dilakukan menggunakan modul regressor pada TensorFlow, Scikit, LGBM dan XGBoost.

In [ ]:
## TENSORFLOW MODEL
class TensorFlow:
  # Inisiasi kelas dan parameter model
  def __init__(self, 
               fitur_train: pd.DataFrame, 
               target_train: pd.DataFrame,
               fitur_test: pd.DataFrame,
               target_test: pd.DataFrame,
               random_seed: int = 11,
               es_patience: int = 0,
               callbacks: list = [],
               loss: Literal["mae", "mse"] = "mae",
               optimizer: Literal["adam", "sgd", "adadelta"] = "adam",
               optimizer_lr: float = 0.01,
               metric: list = ['mean_absolute_error'],
               epoch: int = 100,
               verbose_mode: int = 0):
    self.fitur_train = fitur_train
    self.target_train = target_train
    self.fitur_test = fitur_test
    self.target_test = target_test
    self.random_seed = random_seed
    self.es_patience = es_patience
    self.callbacks = callbacks
    if len(self.callbacks) == 0:
      self.callbacks = [tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10 if self.es_patience <= 0 else self.es_patience)]
    self.optimizer_lr = optimizer_lr
    self.metric = metric
    self.epoch = epoch
    match loss:
      case "mse":
        self.loss = tf.keras.losses.mse
      case _:
        self.loss = tf.keras.losses.mae
    match optimizer:
      case "sgd":
        self.optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=self.optimizer_lr)
      case "adadelta":
        self.optimizer = tf.keras.optimizers.legacy.Adadelta(learning_rate=self.optimizer_lr)
      case _:
        self.optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=self.optimizer_lr)
    self.verbose_mode = verbose_mode
        
  # Compile model
  def compile_model(self, model):
    model.compile(loss=self.loss,
                  optimizer=self.optimizer,
                  metrics=self.metric)
    
  # Fit model
  def fit_model(self, model):
    return model.fit(self.fitur_train, 
                     self.target_train, 
                     epochs=self.epoch, 
                     validation_data=(self.fitur_test, self.target_test), 
                     callbacks=self.callbacks, 
                     verbose=self.verbose_mode)
  
  # Plotting nilai sebenarnya dan prediksi
  def plot_hasil(self):
    return
  
  # Model_Regresi_linear_1_Layer
  def model_dense_1_layer(self,
                          nama_model: str = ""):
    # Set random seed
    tf.random.set_seed = self.random_seed
    # Model Def
    model = tf.keras.Sequential([
      tf.keras.layers.Dense(1, input_dim=X_train.shape[1]) # type: ignore
    ], name=nama_model)
    self.compile_model(model)
    return model
  
  # Model_Regresi_Linear_3_Layer_2_RELU
  def model_dnn_3_layer(self, 
                        unit_1: int = 64, 
                        unit_2: int = 64,
                        nama_model: str = ""):
    # Set random seed
    tf.random.set_seed = self.random_seed
    # Model Def
    model = tf.keras.Sequential([
      tf.keras.layers.Dense(unit_1, activation="relu"),
      tf.keras.layers.Dense(unit_2, activation="relu"),
      tf.keras.layers.Dense(1)
    ], name=nama_model)
    self.compile_model(model)
    return model

In [ ]:
## SCIKIT MODEL
class SK:
  # Inisiasi kelas dan parameter model
  def __init__(self, 
               fitur_train: pd.DataFrame, 
               target_train: pd.DataFrame,
               fitur_test: pd.DataFrame,
               target_test: pd.DataFrame):
    self.fitur_train = fitur_train
    self.target_train = target_train
    self.fitur_test = fitur_test
    self.target_test = target_test
    
  # Fit model
  def fit_model(self, model):
    return model.fit(self.fitur_train, self.target_train)
  
  # Plotting nilai sebenarnya dan prediksi
  def plot_hasil(self):
    return
  
  # Model_SK_Linear_Regresi
  def regresi_linear(self):
    return LinearRegression()
  
  # Model_SK_Decision_Tree
  def decision_tree(self):
    return DecisionTreeRegressor()

In [ ]:
# LIGHTBM DAN XGBOOST MODEL (ENSEMBLE REGRESSOR)
class EnsembleModel:
  def __init__(self,
               fitur_train: pd.DataFrame,
               target_train: pd.DataFrame,
               fitur_test: pd.DataFrame,
               target_test: pd.DataFrame):
    self.fitur_train = fitur_train
    self.target_train = target_train
    self.fitur_test = fitur_test
    self.target_test = target_test
  
  def fit_model(self, model):
    return model.fit(self.fitur_train, self.target_train)
  
  def lgbm(self):
    return lightgbm.LGBMRegressor()
  
  def xgb(self):
    return xgboost.XGBRFRegressor()

### Pelatihan Model

#### TensorFlow

In [ ]:
tensorflow = TensorFlow(
  fitur_train=X_train, # type: ignore
  target_train=y_train, # type: ignore
  fitur_test=X_test, # type: ignore
  target_test=y_test, # type: ignore
  es_patience=20,
  epoch=1000
)

tensorflow_model = [tensorflow.model_dense_1_layer("Model_Dense_1_Layer"), tensorflow.model_dnn_3_layer(nama_model="Model_DNN_3_Layer_RELU_64_64"), tensorflow.model_dnn_3_layer(128, 64, "Model_DNN_3_Layer_RELU_128_64")]

for index, model in enumerate(tensorflow_model):
  # plot_model(model, show_shapes=True)
  # model.summary()
  hasil = tensorflow.fit_model(model)
  if index != 0:
    print("\n")
  print(f"Model {index + 1} Summary dan Mean Absolute Error:\n")
  model.summary()
  print(tf.keras.losses.mean_absolute_error(y_test, model.predict(X_test)))
